<a href="https://colab.research.google.com/github/Hou-jing/Enriching-Pre-trained-Language-Model-with-Entity-Information-for-Relation-Classification/blob/main/R_bert_master_2i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.dropbox.com/s/axxqfdte9e92bwz/train.tsv?dl=0

!wget https://www.dropbox.com/s/1fv24i278rm0dka/test.tsv?dl=0

--2022-02-10 00:01:39--  https://www.dropbox.com/s/axxqfdte9e92bwz/train.tsv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/axxqfdte9e92bwz/train.tsv [following]
--2022-02-10 00:01:40--  https://www.dropbox.com/s/raw/axxqfdte9e92bwz/train.tsv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc34e74cf97fe163772d9cb757b6.dl.dropboxusercontent.com/cd/0/inline/BfaShNN3ErPU4SjshP4VVYMsO7gMubdcstqY5HD9wDeVXUkxiHCUCsOMWBOVEO91ItsSS6XJJjjB8L2DwKpCa5fuMAIbwB4x3eGti8TzexaFPKCiMOTNi0sQuJAuQ8Pk5T6ImS9KkoFeBT2Y59CVfv3g/file# [following]
--2022-02-10 00:01:40--  https://uc34e74cf97fe163772d9cb757b6.dl.dropboxusercontent.com/cd/0/inline/BfaShNN3ErPU4SjshP4VVYMsO7gMubdcstqY5HD9wDeVXUkxiHCUCsOMWBOVEO91ItsSS6XJJjjB8L2DwKpCa5fuMAIbwB

# 新段落

In [ ]:
pip install transformers

     |████████████████████████████████| 3.5 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 50.4 MB/s 
     |████████████████████████████████| 6.8 MB 43.0 MB/s 
     |████████████████████████████████| 596 kB 52.6 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## 尝试2代码

### 数据预处理

In [ ]:
from random import random

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
from transformers import BertModel,BertTokenizer
# model_name='bert-base-chinese'
model_name='bert-base-uncased'
# with open('train.tsv?dl=0' , encoding='utf_8', mode='r') as f:
#     rel2id = {}
#     for line in f:
#         line = line.strip().split('	')
#         if line[0] not in rel2id.keys():
#             rel2id[line[0]] = len(rel2id)
#         else:
#             continue
rel2id={'Other': 0, 'Cause-Effect(e1,e2)': 1, 'Cause-Effect(e2,e1)': 2, 'Instrument-Agency(e1,e2)': 3, 'Instrument-Agency(e2,e1)': 4, 'Product-Producer(e1,e2)': 5, 'Product-Producer(e2,e1)': 6, 'Content-Container(e1,e2)': 7, 'Content-Container(e2,e1)': 8, 'Entity-Origin(e1,e2)': 9, 'Entity-Origin(e2,e1)': 10, 'Entity-Destination(e1,e2)': 11, 'Entity-Destination(e2,e1)': 12, 'Component-Whole(e1,e2)': 13, 'Component-Whole(e2,e1)': 14, 'Member-Collection(e1,e2)': 15, 'Member-Collection(e2,e1)': 16, 'Message-Topic(e1,e2)': 17, 'Message-Topic(e2,e1)': 18}
id2rel={k:v for v,k in rel2id.items()}
class preprocess:
    def read_file(self,file_name):
        with open('%s.tsv?dl=0'%file_name,encoding='utf_8',mode='r') as f:
            rel=[]
            sentence=[]
            max_length=0
            for line in f:
                line=line.strip().split('	')
                rel.append(line[0])
                sent=line[1].replace('</e1>','<e1>').replace('</e2>','<e2>')
                sent='CLS '+sent.replace('<e1>','$').replace('<e2>','#')
                sentence.append(sent)
                if len(sent)>max_length:#max_lenth计算有误，是按单个字母计算的
                    max_length=len(sent)

            rel_id=[]
            for item in rel:
                rel_id.append(rel2id[item])

        return sentence,rel_id,max_length
    def bert_prepro(self,file_name):
        self.bert_pre=BertTokenizer.from_pretrained(model_name)
        train_sents,train_relid,max_length=self.read_file(file_name)
        # print(train_sents[:5],train_relid[:5],rel2id)
        inputs=self.bert_pre(train_sents,add_special_tokens=False,padding=True)#max_length=128,truncation=True
        inputs_id=inputs['input_ids']
        att_mask=inputs['attention_mask']
        if type(inputs_id) != torch.Tensor or type(att_mask) != torch.Tensor:
            att_mask = torch.tensor(att_mask)  # [item.cpu().detach().numpy() for item in train_mask]
            inputs_id = torch.tensor(inputs_id)
        print(type(inputs_id), type(att_mask))
        return inputs_id,att_mask,train_relid,max_length

    def entity_tensor(self):
        head_id = self.bert_pre.encode('$', return_tensors='pt', padding=False, add_special_tokens=False)
        tail_id = self.bert_pre.encode('#', return_tensors='pt', add_special_tokens=False)
        head_id=int(head_id[0])
        tail_id=int(tail_id[0])
        return head_id,tail_id#plan to find the entity vector
    def entity_mask(self,file_name):
        inputs_id,att_mask,train_relid,max_length=self.bert_prepro(file_name)
        head_id,tail_id=self.entity_tensor()
        head_entity_mask = torch.zeros((inputs_id.size()[0], inputs_id.size()[1]))
        tail_entity_mask = torch.zeros((inputs_id.size()[0], inputs_id.size()[1]))
        for i in range(inputs_id.size(0)):
            head_pos = (inputs_id[i].squeeze() == head_id).nonzero().squeeze()
            tail_pos = (inputs_id[i].squeeze() == tail_id).nonzero().squeeze()#位置index
            for pos in range(int(head_pos[0]) + 1, int(head_pos[1])):
                head_entity_mask[i,pos]=1
            for pos in range(int(tail_pos[0]) + 1, int(tail_pos[1])):
                tail_entity_mask[i, pos] = 1
        return head_entity_mask.unsqueeze(-1),tail_entity_mask.unsqueeze(-1)#(增加一个维度）
if __name__=='__main__':
    file_type=['train','test']
    prepro=preprocess()
    for ftype in file_type:
        if ftype=='train':
            train_id,train_mask,train_relid,max_length=prepro.bert_prepro(ftype)
            train_head_mask,train_tail_mask=prepro.entity_mask(ftype)
        # else:
        #     test_id,test_mask,test_relid,max_length=prepro.bert_prepro(ftype)
        #     head_mask,tail_mask=prepro.entity_mask(ftype)
    head_id,tail_id=prepro.entity_tensor()

<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
def same_seeds(seed):
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
        # np.random.seed(seed)
        # random.seed(seed)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = True
same_seeds(0)

### Model 1

In [ ]:
#@title

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.label_num = len(rel2id)
        self.encode = BertModel.from_pretrained(model_name)
        self.fc = nn.Sequential(
            nn.Dropout(0.1),
            nn.Tanh(),
            nn.Linear(768, 128),
        )
        self.fc2 = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(384, 64),#128*3
            nn.Linear(64, self.label_num),
            nn.Softmax(1)
        )
    def bert_model(self,inputs_id,att_mask,head_mask,tail_mask):
        bert_output=self.encode(inputs_id,att_mask)[0]
        cls=bert_output[:,0,:]
        sent_vector=bert_output[:,1:,:]
        head_entity_aver=(sent_vector*head_mask).mean(1)#乘法有错误(batchsize,seq_len,768),(batchsize,seq_len)
        tail_entity_aver=(sent_vector*tail_mask).mean(1)
        return cls,head_entity_aver,tail_entity_aver
    def forward(self,inputs_id,att_mask,head_mask,tail_mask):
        cls,head_entity_aver,tail_entity_aver= self.bert_model(inputs_id,att_mask,head_mask,tail_mask)
        cls,head_entity_aver,tail_entity_aver=cls.to(device),head_entity_aver.to(device),tail_entity_aver.to(device)
        batch_tensor=torch.cat((self.fc(cls),self.fc(head_entity_aver),self.fc(tail_entity_aver)),dim=1)
        return self.fc2(batch_tensor)


### Model 2

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.label_num = len(rel2id)
        self.encode = BertModel.from_pretrained(model_name)
        self.tanh=nn.Tanh()
        self.fc_cls = nn.Sequential(
            nn.Dropout(0.1),
            nn.Tanh(),
            nn.Linear(768, 128),
        )
        self.fc_en1 = nn.Sequential(
            nn.Dropout(0.1),
            nn.Tanh(),
            nn.Linear(768, 128),
        )
        self.fc_en2 = nn.Sequential(
            nn.Dropout(0.1),
            nn.Tanh(),
            nn.Linear(768, 128),
        )
        self.fc2 = nn.Sequential(
            nn.Dropout(0.1),
            # nn.Linear(384, 64),#128*3
            nn.Tanh(),
            nn.Linear(384,self.label_num )
        )

        self.s=nn.Softmax()
    def forward(self,inputs_id,att_mask,head_mask,tail_mask):
        sent_vector=self.encode(inputs_id,att_mask)[0]
        cls=sent_vector[:,0,:]
        head_entity_aver=(sent_vector*head_mask).mean(1)#乘法有错误(batchsize,seq_len,768),(batchsize,seq_len)
        tail_entity_aver=(sent_vector*tail_mask).mean(1)
        cls=self.tanh(cls)
        head_entity_aver=self.tanh(head_entity_aver)
        tail_entity_aver=self.tanh(tail_entity_aver)
        # cls,head_entity_aver,tail_entity_aver=cls.to(device),head_entity_aver.to(device),tail_entity_aver.to(device)
        batch_tensor=torch.cat((self.fc_cls(cls),self.fc_en1(head_entity_aver),self.fc_en2(tail_entity_aver)),dim=1)
        batch_tensor=self.tanh(batch_tensor)
        batch_tensor=self.fc2(batch_tensor)
        return batch_tensor

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.label_num = len(rel2id)
        self.encode = BertModel.from_pretrained(model_name)
        self.tanh=nn.Tanh()
        self.fc_cls = nn.Sequential(
            # nn.Dropout(0.1),
            nn.Tanh(),
            nn.Linear(768*3, 128*3))
        self.fc2 = nn.Sequential(
            # nn.Dropout(0.1),
            # nn.Linear(384, 64),#128*3
            nn.ReLU(),
            nn.Linear(384, 1),
        )

        self.s=nn.Softmax()
    def forward(self,inputs_id,att_mask,head_mask,tail_mask):
        sent_vector=self.encode(inputs_id,att_mask)[0]
        cls=sent_vector[:,0,:]
        head_entity_aver=(sent_vector*head_mask).mean(1)#乘法有错误(batchsize,seq_len,768),(batchsize,seq_len)
        tail_entity_aver=(sent_vector*tail_mask).mean(1)
        cls=self.tanh(cls)
        head_entity_aver=self.tanh(head_entity_aver)
        tail_entity_aver=self.tanh(tail_entity_aver)
        cls,head_entity_aver,tail_entity_aver=cls.to(device),head_entity_aver.to(device),tail_entity_aver.to(device)
        batch_tensor=torch.cat((cls,head_entity_aver,tail_entity_aver),dim=1)
        batch_tensor=self.fc_cls(batch_tensor)
        batch_tensor=self.fc2(batch_tensor)
        return batch_tensor

In [ ]:
model=Model()

# model.bert_model(torch.from_numpy(np.array(train_id[:4])),torch.from_numpy(np.array(train_mask[:4])))

criterier=nn.MSELoss()
optim=torch.optim.Adam(model.parameters(),lr=1e-3)

from torch.utils.data import TensorDataset,DataLoader
train_label=torch.tensor(train_relid)#转为tensor形式
print(type(train_label),type(train_mask),type(train_label))
if type(train_id)!=torch.Tensor or type(train_mask)!=torch.Tensor:
    train_mask=torch.tensor(train_mask)#[item.cpu().detach().numpy() for item in train_mask]
    train_id=torch.tensor(train_id)
print(type(train_id),type(train_mask),type(train_label))
train_set=TensorDataset(train_id,train_mask,train_head_mask,train_tail_mask,train_label)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
#@title
EPOCHS=5
# for epoch in range(EPOCHS):
#     step=1
#     model.train()
#     for i,data in tqdm(enumerate(train_loader)):
#         input_id,input_mask,label=data
#         pre=model.forward2(input_id,input_mask)
#         loss=criterier(pre,label)
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.empty_cache()
def train(net,dataset,num_epochs, learning_rate,  batch_size):
    i=1
    print('执行次数为：{}'.format(i))
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    net=net.to(device)
    net.train()
    optimizer =torch.optim.Adam(net.parameters(), lr=learning_rate)
    #optimizer = AdamW(net.parameters(), lr=learning_rate)
    criterier=nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
      
        train_loss, train_acc = 0, 0
        accum_iter=4
        step=1
        for batch_idx,data in enumerate(tqdm(train_loader)):
          text,mask,head_mask,tail_mask,label=data
          text,mask,head_mask,tail_mask,label=text.to(device),mask.to(device),head_mask.to(device),tail_mask.to(device),label.to(device)
          optimizer.zero_grad()
            # with torch.set_grad_enabled(True):
          pre = net(text,mask,head_mask,tail_mask)
            # label=label.reshape(batch_size,1)
            # label=label.to(torch.float)
          label=label.squeeze()
          label = label.long()
          loss = criterier(pre, label)
          loss.backward(retain_graph=True)
          optimizer.step()
          _, predicted = torch.max(pre, 1)
          train_acc += (predicted.cpu() == label.cpu()).sum().item()
          train_loss += loss.item()
          step+=1
          if step%100==0:
            print('train_epoch|{},acc={},loss={}'.format(epoch,train_acc/(int(step/100)*batch_size*100),train_loss/(int(step/100)*100)))


id2rel={k:v for v,k in rel2id.items()}

In [ ]:
from sklearn.metrics import accuracy_score
y_true=train_relid

### 模型训练

In [ ]:
import transformers
# for epoch in range(EPOCHS):
#     step=1
#     model.train()
#     for i,data in tqdm(enumerate(train_loader)):
#         input_id,input_mask,label=data
#         pre=model.forward2(input_id,input_mask)
#         loss=criterier(pre,label)
device='cuda' if torch.cuda.is_available() else 'cpu'


def train(net,dataset,num_epochs, learning_rate,  batch_size):
    i=1
    print('执行次数为：{}'.format(i))
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    t_total = len(train_loader) //num_epochs
    optimizer =torch.optim.Adam(net.parameters(), lr=learning_rate)
    # scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=t_total)
    
    net=net.to(device)
    net.train()   
    #optimizer = AdamW(net.parameters(), lr=learning_rate)
    criterier=nn.CrossEntropyLoss()
    # criterier=nn.MSELoss()
    for epoch in range(num_epochs):
      predict=[]
      train_loss, train_acc = 0, 0
      accum_iter=4
      step=1
      for batch_idx,data in enumerate(train_loader):
        text,mask,head_mask,tail_mask,label=data
        text,mask,head_mask,tail_mask,label=text.to(device),mask.to(device),head_mask.to(device),tail_mask.to(device),label.to(device)
        optimizer.zero_grad()
            # with torch.set_grad_enabled(True):
        pre = net(text,mask,head_mask,tail_mask)
            # label=label.reshape(batch_size,1)
            # label=label.to(torch.float)
        label=label.squeeze()
        label = label.long()
        # label = label.float()
        loss = criterier(pre, label)
        loss.backward()
  
        optimizer.step()
        # scheduler.step()
        _, predicted = torch.max(pre, 1)
        predict.extend(predicted.cpu().tolist())
        train_acc += (predicted.cpu() == label.cpu()).sum().item()
        train_loss += loss.item()

        step+=1
        if step%100==0:
          print('train_epoch|{},acc={},loss={}'.format(epoch,train_acc/(int(step/100)*batch_size*100),train_loss/(int(step/100)*100)))
      acc1=accuracy_score(y_true,predict,normalize=True)

      print('{},{}'.format(epoch,acc1))

id2rel={k:v for v,k in rel2id.items()}
EPOCHS=5
train(model,train_set,EPOCHS,learning_rate=2e-5,batch_size=16)

执行次数为：1
train_epoch|0,acc=0.183125,loss=2.7229765963554384
train_epoch|0,acc=0.298125,loss=2.5699595767259598
train_epoch|0,acc=0.393125,loss=2.414390085140864
train_epoch|0,acc=0.46671875,loss=2.275409690737724
train_epoch|0,acc=0.519625,loss=2.161467492341995
0,0.105375
train_epoch|1,acc=0.80375,loss=1.463382865190506
train_epoch|1,acc=0.8025,loss=1.4290784987807275
train_epoch|1,acc=0.801875,loss=1.3954670498768489
train_epoch|1,acc=0.80734375,loss=1.3486649517714977
train_epoch|1,acc=0.810375,loss=1.3105094010829925
1,0.0825
train_epoch|2,acc=0.875625,loss=0.9573589152097702
train_epoch|2,acc=0.8778125,loss=0.9350774291157723
train_epoch|2,acc=0.879375,loss=0.9153783547878266
train_epoch|2,acc=0.8784375,loss=0.9037987418472767
train_epoch|2,acc=0.8755,loss=0.8972366044521332
2,0.08675
train_epoch|3,acc=0.895625,loss=0.7054354539513588
train_epoch|3,acc=0.9128125,loss=0.6613687165081501
train_epoch|3,acc=0.9122916666666666,loss=0.6592493134737015
train_epoch|3,acc=0.91140625,loss=0.

In [ ]:
EPOCHS=20
train(model,train_set,EPOCHS,learning_rate=1e-3,batch_size=16)

### test 数据预处理

In [ ]:

class preprocess:
    def read_file(self,file_name):
        with open('%s.tsv?dl=0'%file_name,encoding='utf_8',mode='r') as f:
            rel=[]
            sentence=[]
            max_length=0
            for line in f:
                line=line.strip().split('	')
                rel.append(line[0])
                sent=line[1].replace('</e1>','<e1>').replace('</e2>','<e2>')
                sent='CLS '+sent.replace('<e1>','$').replace('<e2>','#')
                sentence.append(sent)
                if len(sent)>max_length:#max_lenth计算有误，是按单个字母计算的
                    max_length=len(sent)

            rel_id=[]
            for item in rel:
                rel_id.append(rel2id[item])

        return sentence,rel_id,max_length
    def bert_prepro(self,file_name):
        self.bert_pre=BertTokenizer.from_pretrained(model_name)
        train_sents,train_relid,max_length=self.read_file(file_name)
        # print(train_sents[:5],train_relid[:5],rel2id)
        inputs=self.bert_pre(train_sents,add_special_tokens=False,padding=True)#max_length=128,truncation=True
        inputs_id=inputs['input_ids']
        att_mask=inputs['attention_mask']
        if type(inputs_id) != torch.Tensor or type(att_mask) != torch.Tensor:
            att_mask = torch.tensor(att_mask)  # [item.cpu().detach().numpy() for item in train_mask]
            inputs_id = torch.tensor(inputs_id)
        print(type(inputs_id), type(att_mask))
        return inputs_id,att_mask,train_relid,max_length

    def entity_tensor(self):
        head_id = self.bert_pre.encode('$', return_tensors='pt', padding=False, add_special_tokens=False)
        tail_id = self.bert_pre.encode('#', return_tensors='pt', add_special_tokens=False)
        head_id=int(head_id[0])
        tail_id=int(tail_id[0])
        return head_id,tail_id#plan to find the entity vector
    def entity_mask(self,file_name):
        inputs_id,att_mask,train_relid,max_length=self.bert_prepro(file_name)
        head_id,tail_id=self.entity_tensor()
        head_entity_mask = torch.zeros((inputs_id.size()[0], inputs_id.size()[1]))
        tail_entity_mask = torch.zeros((inputs_id.size()[0], inputs_id.size()[1]))
        for i in range(inputs_id.size(0)):
            head_pos = (inputs_id[i].squeeze() == head_id).nonzero().squeeze()
            tail_pos = (inputs_id[i].squeeze() == tail_id).nonzero().squeeze()#位置index
            for pos in range(int(head_pos[0]) + 1, int(head_pos[1])):
                head_entity_mask[i,pos]=1
            for pos in range(int(tail_pos[0]) + 1, int(tail_pos[1])):
                tail_entity_mask[i, pos] = 1
        return head_entity_mask.unsqueeze(-1),tail_entity_mask.unsqueeze(-1)#(增加一个维度）
if __name__=='__main__':
    file_type=['test']
    prepro=preprocess()
    for ftype in file_type:
        if ftype=='train':
            train_id,train_mask,train_relid,max_length=prepro.bert_prepro(ftype)
            train_head_mask,train_tail_mask=prepro.entity_mask(ftype)
        else:
            test_id,test_mask,test_relid,max_length=prepro.bert_prepro(ftype)
            head_mask,tail_mask=prepro.entity_mask(ftype)
    head_id,tail_id=prepro.entity_tensor()

<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>


### 模型测试

In [ ]:
def model_test(net,dataset,num_epochs, learning_rate,  batch_size):
    net.to(device)
    net.eval()
    # rel_list = []
    correct=0
    total=0
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    with torch.no_grad():
        result=[]
        for batch_idx, data in enumerate(tqdm(train_loader)):
            text,mask,head_mask,tail_mask=data
            text,mask,head_mask,tail_mask=text.to(device),mask.to(device),head_mask.to(device),tail_mask.to(device)
            pre = net(text,mask,head_mask,tail_mask)
            # label=label.reshape(batch_size,1)
            # label=label.to(torch.float)
            _, predicted = torch.max(pre, 1)
            result.extend(predicted.cpu().detach().tolist())
        #     correct += (predicted.cpu() == label.cpu()).sum().item()
        # print('test_acc={}'.format(correct /len(train_loader)*4))
    # rel_list.append(id2rel[result])
    return result
train_set=TensorDataset(test_id,test_mask,head_mask,tail_mask)
rel_list=model_test(model,train_set,EPOCHS,learning_rate=1e-3,batch_size=4)

100%|██████████| 680/680 [00:11<00:00, 60.72it/s]


### 准确率和召回率计算

In [ ]:
print(rel_list[:10])

[17, 6, 4, 11, 2, 13, 0, 0, 0, 17]


In [ ]:
print(test_relid[:10])

[17, 6, 4, 11, 2, 13, 5, 16, 13, 17]


In [ ]:
acc1=accuracy_score(test_relid,rel_list,normalize=True)
print(acc1)

0.8292234081707766


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import recall_score

rel1=recall_score(test_relid,rel_list,average='micro')
print(rel1)

0.8292234081707766


In [ ]:
pre, rec, f1, sup = precision_recall_fscore_support(test_relid,rel_list)
print("precision:", pre, "\nrecall:", rec, "\nf1-score:", f1, "\nsupport:", sup)

precision: [0.65270936 0.92753623 0.90547264 0.8        0.68156425 0.83809524
 0.82170543 0.92537313 0.88571429 0.87019231 0.95348837 0.9
 0.         0.91724138 0.7615894  0.78571429 0.85321101 0.90140845
 0.77966102] 
recall: [0.58370044 0.95522388 0.93814433 0.54545455 0.91044776 0.81481481
 0.86178862 0.81045752 0.79487179 0.85781991 0.87234043 0.95876289
 0.         0.82098765 0.76666667 0.6875     0.92537313 0.91428571
 0.90196078] 
f1-score: [0.61627907 0.94117647 0.92151899 0.64864865 0.77955272 0.82629108
 0.84126984 0.8641115  0.83783784 0.86396181 0.91111111 0.92845258
 0.         0.86644951 0.7641196  0.73333333 0.88782816 0.90780142
 0.83636364] 
support: [454 134 194  22 134 108 123 153  39 211  47 291   1 162 150  32 201 210
  51]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Bert_NO_SEQ
vector中学习speical token的encoding_

In [ ]:
#第二种配置是丢弃特定的单独标记（即“$”和“#”）
class preprocess:
    def read_file(self,file_name):
        with open('%s.tsv?dl=0'%file_name,encoding='utf_8',mode='r') as f:
            rel=[]
            sentence=[]
            max_length=0
            for line in f:
                line=line.strip().split('	')
                rel.append(line[0])
                sent=line[1].replace('</e1>','<e1>').replace('</e2>','<e2>')
                sent='CLS '+sent.replace('<e1>','$').replace('<e2>','#')
                sentence.append(sent)
                if len(sent)>max_length:#max_lenth计算有误，是按单个字母计算的
                    max_length=len(sent)

            rel_id=[]
            for item in rel:
                rel_id.append(rel2id[item])

        return sentence,rel_id,max_length
    def bert_prepro(self,file_name):
        self.bert_pre=BertTokenizer.from_pretrained(model_name)
        train_sents,train_relid,max_length=self.read_file(file_name)
        # print(train_sents[:5],train_relid[:5],rel2id)
        inputs=self.bert_pre(train_sents,add_special_tokens=False,padding=True)#max_length=128,truncation=True
        inputs_id=inputs['input_ids']
        att_mask=inputs['attention_mask']
        if type(inputs_id) != torch.Tensor or type(att_mask) != torch.Tensor:
            att_mask = torch.tensor(att_mask)  # [item.cpu().detach().numpy() for item in train_mask]
            inputs_id = torch.tensor(inputs_id)
        print(type(inputs_id), type(att_mask))
        return inputs_id,att_mask,train_relid,max_length

    def entity_tensor(self):
        head_id = self.bert_pre.encode('$', return_tensors='pt', padding=False, add_special_tokens=False)
        tail_id = self.bert_pre.encode('#', return_tensors='pt', add_special_tokens=False)
        head_id=int(head_id[0])
        tail_id=int(tail_id[0])
        return head_id,tail_id#plan to find the entity vector
    def entity_mask(self,file_name):
        inputs_id,att_mask,train_relid,max_length=self.bert_prepro(file_name)
        head_id,tail_id=self.entity_tensor()
        head_entity_mask = torch.zeros((inputs_id.size()[0], inputs_id.size()[1]))
        tail_entity_mask = torch.zeros((inputs_id.size()[0], inputs_id.size()[1]))
        for i in range(inputs_id.size(0)):
            head_pos = (inputs_id[i].squeeze() == head_id).nonzero().squeeze()
            tail_pos = (inputs_id[i].squeeze() == tail_id).nonzero().squeeze()#位置index
            for pos in range(int(head_pos[0]), int(head_pos[1])+1):
                head_entity_mask[i,pos]=1
            for pos in range(int(tail_pos[0]), int(tail_pos[1])+1):
                tail_entity_mask[i, pos] = 1
        return head_entity_mask.unsqueeze(-1),tail_entity_mask.unsqueeze(-1)#(增加一个维度）
if __name__=='__main__':
    file_type=['train','test']
    prepro=preprocess()
    for ftype in file_type:
        if ftype=='train':
            train_id,train_mask,train_relid,max_length=prepro.bert_prepro(ftype)
            train_head_mask,train_tail_mask=prepro.entity_mask(ftype)

<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>


### model

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.label_num = len(rel2id)
        self.encode = BertModel.from_pretrained(model_name)
        self.tanh=nn.Tanh()
        self.fc_cls = nn.Sequential(
            nn.Dropout(0.1),
            nn.Tanh(),
            nn.Linear(768, 128),
        )
        self.fc_en1 = nn.Sequential(
            nn.Dropout(0.1),
            nn.Tanh(),
            nn.Linear(768, 128),
        )
        self.fc_en2 = nn.Sequential(
            nn.Dropout(0.1),
            nn.Tanh(),
            nn.Linear(768, 128),
        )
        self.fc2 = nn.Sequential(
            nn.Dropout(0.1),
            # nn.Linear(384, 64),#128*3
            nn.Tanh(),
            nn.Linear(384,self.label_num )
        )

        self.s=nn.Softmax()
    def forward(self,inputs_id,att_mask,head_mask,tail_mask):
        sent_vector=self.encode(inputs_id,att_mask)[0]
        cls=sent_vector[:,0,:]
        head_entity_aver=(sent_vector*head_mask).mean(1)#乘法有错误(batchsize,seq_len,768),(batchsize,seq_len)
        tail_entity_aver=(sent_vector*tail_mask).mean(1)
        cls=self.tanh(cls)
        head_entity_aver=self.tanh(head_entity_aver)
        tail_entity_aver=self.tanh(tail_entity_aver)
        # cls,head_entity_aver,tail_entity_aver=cls.to(device),head_entity_aver.to(device),tail_entity_aver.to(device)
        batch_tensor=torch.cat((self.fc_cls(cls),self.fc_en1(head_entity_aver),self.fc_en2(tail_entity_aver)),dim=1)
        batch_tensor=self.tanh(batch_tensor)
        batch_tensor=self.fc2(batch_tensor)
        return batch_tensor

In [ ]:
model=Model()

# model.bert_model(torch.from_numpy(np.array(train_id[:4])),torch.from_numpy(np.array(train_mask[:4])))

criterier=nn.MSELoss()
optim=torch.optim.Adam(model.parameters(),lr=1e-3)

from torch.utils.data import TensorDataset,DataLoader
train_label=torch.tensor(train_relid)#转为tensor形式
print(type(train_label),type(train_mask),type(train_label))
if type(train_id)!=torch.Tensor or type(train_mask)!=torch.Tensor:
    train_mask=torch.tensor(train_mask)#[item.cpu().detach().numpy() for item in train_mask]
    train_id=torch.tensor(train_id)
print(type(train_id),type(train_mask),type(train_label))
train_set=TensorDataset(train_id,train_mask,train_head_mask,train_tail_mask,train_label)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


### train

In [ ]:
import transformers
# for epoch in range(EPOCHS):
#     step=1
#     model.train()
#     for i,data in tqdm(enumerate(train_loader)):
#         input_id,input_mask,label=data
#         pre=model.forward2(input_id,input_mask)
#         loss=criterier(pre,label)
device='cuda' if torch.cuda.is_available() else 'cpu'


def train(net,dataset,num_epochs, learning_rate,  batch_size):
    i=1
    print('执行次数为：{}'.format(i))
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    t_total = len(train_loader) //num_epochs
    optimizer =torch.optim.Adam(net.parameters(), lr=learning_rate)
    # scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=t_total)
    
    net=net.to(device)
    net.train()   
    #optimizer = AdamW(net.parameters(), lr=learning_rate)
    criterier=nn.CrossEntropyLoss()
    # criterier=nn.MSELoss()
    for epoch in range(num_epochs):
      predict=[]
      train_loss, train_acc = 0, 0
      accum_iter=4
      step=1
      for batch_idx,data in enumerate(train_loader):
        text,mask,head_mask, tail_mask, label=data
        text,mask,head_mask,tail_mask,label=text.to(device),mask.to(device),head_mask.to(device),tail_mask.to(device),label.to(device)
        optimizer.zero_grad()
            # with torch.set_grad_enabled(True):
        pre = net(text,mask,head_mask,tail_mask)
            # label=label.reshape(batch_size,1)
            # label=label.to(torch.float)
        label=label.squeeze()
        label = label.long()
        # label = label.float()
        loss = criterier(pre, label)
        loss.backward()
  
        optimizer.step()
        # scheduler.step()
        _, predicted = torch.max(pre, 1)
        predict.extend(predicted.cpu().tolist())
        train_acc += (predicted.cpu() == label.cpu()).sum().item()
        train_loss += loss.item()

        step+=1
        if step%100==0:
          print('train_epoch|{},acc={},loss={}'.format(epoch,train_acc/(int(step/100)*batch_size*100),train_loss/(int(step/100)*100)))
      acc1=accuracy_score(y_true,predict,normalize=True)

      print('{},{}'.format(epoch,acc1))

id2rel={k:v for v,k in rel2id.items()}
EPOCHS=10
train(model,train_set,EPOCHS,learning_rate=2e-5,batch_size=16)

执行次数为：1
train_epoch|0,acc=0.166875,loss=2.766641778945923
train_epoch|0,acc=0.2128125,loss=2.6897992134094237
train_epoch|0,acc=0.29270833333333335,loss=2.55629812280337
train_epoch|0,acc=0.36125,loss=2.4308526581525802
train_epoch|0,acc=0.425375,loss=2.3031116523742674
0,0.124625
train_epoch|1,acc=0.7375,loss=1.594305429458618
train_epoch|1,acc=0.7559375,loss=1.5239644068479539
train_epoch|1,acc=0.768125,loss=1.459875106215477
train_epoch|1,acc=0.77390625,loss=1.4165464161336423
train_epoch|1,acc=0.7835,loss=1.3669062204360962
1,0.078
train_epoch|2,acc=0.869375,loss=0.9447288227081299
train_epoch|2,acc=0.869375,loss=0.934832047522068
train_epoch|2,acc=0.8670833333333333,loss=0.9180077483256658
train_epoch|2,acc=0.86515625,loss=0.9067925771325827
train_epoch|2,acc=0.864375,loss=0.8848348079919816
2,0.0855
train_epoch|3,acc=0.90125,loss=0.6531433871388436
train_epoch|3,acc=0.905625,loss=0.6389143994450569
train_epoch|3,acc=0.9095833333333333,loss=0.6234542191028595
train_epoch|3,acc=0.9

### test

In [ ]:

class preprocess:
    def read_file(self,file_name):
        with open('%s.tsv?dl=0'%file_name,encoding='utf_8',mode='r') as f:
            rel=[]
            sentence=[]
            max_length=0
            for line in f:
                line=line.strip().split('	')
                rel.append(line[0])
                sent=line[1].replace('</e1>','<e1>').replace('</e2>','<e2>')
                sent='CLS '+sent.replace('<e1>','$').replace('<e2>','#')
                sentence.append(sent)
                if len(sent)>max_length:#max_lenth计算有误，是按单个字母计算的
                    max_length=len(sent)

            rel_id=[]
            for item in rel:
                rel_id.append(rel2id[item])

        return sentence,rel_id,max_length
    def bert_prepro(self,file_name):
        self.bert_pre=BertTokenizer.from_pretrained(model_name)
        train_sents,train_relid,max_length=self.read_file(file_name)
        # print(train_sents[:5],train_relid[:5],rel2id)
        inputs=self.bert_pre(train_sents,add_special_tokens=False,padding=True)#max_length=128,truncation=True
        inputs_id=inputs['input_ids']
        att_mask=inputs['attention_mask']
        if type(inputs_id) != torch.Tensor or type(att_mask) != torch.Tensor:
            att_mask = torch.tensor(att_mask)  # [item.cpu().detach().numpy() for item in train_mask]
            inputs_id = torch.tensor(inputs_id)
        print(type(inputs_id), type(att_mask))
        return inputs_id,att_mask,train_relid,max_length

    def entity_tensor(self):
        head_id = self.bert_pre.encode('$', return_tensors='pt', padding=False, add_special_tokens=False)
        tail_id = self.bert_pre.encode('#', return_tensors='pt', add_special_tokens=False)
        head_id=int(head_id[0])
        tail_id=int(tail_id[0])
        return head_id,tail_id#plan to find the entity vector
    def entity_mask(self,file_name):
        inputs_id,att_mask,train_relid,max_length=self.bert_prepro(file_name)
        head_id,tail_id=self.entity_tensor()
        head_entity_mask = torch.zeros((inputs_id.size()[0], inputs_id.size()[1]))
        tail_entity_mask = torch.zeros((inputs_id.size()[0], inputs_id.size()[1]))
        for i in range(inputs_id.size(0)):
            head_pos = (inputs_id[i].squeeze() == head_id).nonzero().squeeze()
            tail_pos = (inputs_id[i].squeeze() == tail_id).nonzero().squeeze()#位置index
            for pos in range(int(head_pos[0]) + 1, int(head_pos[1])):
                head_entity_mask[i,pos]=1
            for pos in range(int(tail_pos[0]) + 1, int(tail_pos[1])):
                tail_entity_mask[i, pos] = 1
        return head_entity_mask.unsqueeze(-1),tail_entity_mask.unsqueeze(-1)#(增加一个维度）
if __name__=='__main__':
    file_type=['test']
    prepro=preprocess()
    for ftype in file_type:
        if ftype=='train':
            train_id,train_mask,train_relid,max_length=prepro.bert_prepro(ftype)
            train_head_mask,train_tail_mask=prepro.entity_mask(ftype)
        else:
            test_id,test_mask,test_relid,max_length=prepro.bert_prepro(ftype)
            head_mask,tail_mask=prepro.entity_mask(ftype)
    head_id,tail_id=prepro.entity_tensor()

<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
def model_test(net,dataset,num_epochs, learning_rate,  batch_size):
    net.to(device)
    net.eval()
    # rel_list = []
    correct=0
    total=0
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    with torch.no_grad():
        result=[]
        for batch_idx, data in enumerate(tqdm(train_loader)):
            text,mask,head_mask,tail_mask=data
            text,mask,head_mask,tail_mask=text.to(device),mask.to(device),head_mask.to(device),tail_mask.to(device)
            pre = net(text,mask,head_mask,tail_mask)
            # label=label.reshape(batch_size,1)
            # label=label.to(torch.float)
            _, predicted = torch.max(pre, 1)
            result.extend(predicted.cpu().detach().tolist())
        #     correct += (predicted.cpu() == label.cpu()).sum().item()
        # print('test_acc={}'.format(correct /len(train_loader)*4))
    # rel_list.append(id2rel[result])
    return result
train_set=TensorDataset(test_id,test_mask,head_mask,tail_mask)
rel_list=model_test(model,train_set,EPOCHS,learning_rate=1e-3,batch_size=4)

100%|██████████| 680/680 [00:10<00:00, 62.18it/s]


In [ ]:
print(test_relid[:10])
rel_list[:10]

[17, 6, 4, 11, 2, 13, 5, 16, 13, 17]


[17, 6, 4, 11, 2, 13, 0, 0, 13, 17]

### acc&&F1

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
acc1=accuracy_score(test_relid,rel_list,normalize=True)
print(acc1)
from sklearn.metrics import recall_score

rel1=recall_score(test_relid,rel_list,average='micro')
print(rel1)

0.8244387191755613
0.8244387191755613


In [ ]:
rel1=recall_score(test_relid,rel_list,average='macro')
print(rel1)

0.7801739304822566
